## imports

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split

## Parameters

In [50]:
# Define hyperparameters
input_size = 1
hidden_size = 128
num_layers = 1
learning_rate = 0.001
num_epochs = 100

## Data

In [51]:

# Load the CSV data
data = pd.read_csv('data.csv')

# Define input and output
X1 = data['num1'].values.reshape(-1, 1)
X2 = data['num2'].values.reshape(-1, 1)
y = data['label'].values

## Twin Rnn Model

In [52]:
# Define a custom LSTM model
class TwinLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(TwinLSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers)
        self.lstm2 = nn.LSTM(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size*2, 1)

    def forward(self, x1, x2):
        _, (hidden1, _) = self.lstm1(x1)
        _, (hidden2, _) = self.lstm2(x2)
        combined_hidden = torch.cat((hidden1[-1], hidden2[-1]), dim=1)
        output = torch.sigmoid(self.fc(combined_hidden))
        return output



In [60]:
X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(X1, X2, y, test_size=0.2, random_state=42)

X1_train = torch.tensor(X1_train, dtype=torch.float32).unsqueeze(2)
X2_train = torch.tensor(X2_train, dtype=torch.float32).unsqueeze(2)
y_train = torch.tensor(y_train, dtype=torch.float32) 
X1_test = torch.tensor(X1_test, dtype=torch.float32)
X2_test = torch.tensor(X2_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32) 

model = TwinLSTM(input_size, hidden_size, num_layers)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Training


In [61]:
# Training the model
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X1_train, X2_train)
    loss = criterion(outputs, y_train)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


ValueError: Using a target size (torch.Size([320])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.

## Evaluate

In [30]:

# Testing the model
with torch.no_grad():
    outputs = model(X1_test.view(-1, 1, 1), X2_test.view(-1, 1, 1))
    predicted = (outputs.view(-1) > 0.5).float()
    accuracy = (predicted == y_test).float().mean()
    print(f'Accuracy: {accuracy.item()*100:.2f}%')

Accuracy: 70.00%
